<a href="https://colab.research.google.com/github/Omo-Tines/Ideal-Number-Classifiction-using-Cross-Entropy-Loss-ML/blob/main/ML_LAB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
#importing the various libraries needed
from fastai.vision.all import *
from fastbook import *
import torch
import torch.nn as tt
import torchvision
import torchvision.transforms as transforms

Mounted at /content/gdrive


In [ ]:
#part1
m= tensor([1,3,6])
a= tensor([4,7,9],[9,7,5])

print(m.expand_as(a)),print(a.stride(0))

tensor([[1, 3, 6],
        [1, 3, 6]])
3


(None, None)

Using the untar_data we can download the dataset and store it in a variable path

In [ ]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path

Declaring my hyperparameters;

i made use of 784 for the length_input because of the image size of 28x28 and the flattening to a 1d array and the rest where chosen at random

In [ ]:
length_input = 784
length_hidden = 120
num_digits = 4
#hidd_2= 40
runtime = 4 #number of iterations for training
batch_length = 100
learning_rate = 0.0001

In [ ]:
one_set = torchvision.datasets.MNIST(root=(path/'training'/'1'),train=True,transform=transforms.ToTensor(),download=True)
three_set = torchvision.datasets.MNIST(root=(path/'training'/'3'),train=True,transform=transforms.ToTensor(),download=True)
sev_set = torchvision.datasets.MNIST(root=(path/'training'/'7'),train=True,transform=transforms.ToTensor(),download=True)
eight_set = torchvision.datasets.MNIST(root=(path/'training'/'8'),train=True,transform=transforms.ToTensor(),download=True)



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.fastai/data/mnist_png/training/1/MNIST/raw/train-images-idx3-ubyte.gz to /root/.fastai/data/mnist_png/training/1/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.fastai/data/mnist_png/training/1/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.fastai/data/mnist_png/training/1/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting /root/.fastai/data/mnist_png/training/8/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.fastai/data/mnist_png/training/8/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.fastai/data/mnist_png/training/8/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.fastai/data/mnist_png/training/8/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.fastai/data/mnist_png/training/8/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.fastai/data/mnist_png/training/8/MNIST/raw



In [ ]:
#Passing all the Training sets into one
#concatenating the training sets
total_train = one_set+three_set+sev_set+eight_set
#turning it into tensor format
train_loader = torch.utils.data.DataLoader(dataset=total_train,batch_size=batch_length,shuffle=True)

In [ ]:
#creating the neural network
#define the init and forward methods
class feedfwdnet(tt.Module):
  def __init__(self,length_input,length_hidden,num_digits):
    super(feedfwdnet,self).__init__()
    self.inputs= length_input
    self.layers = tt.Sequential(
      tt.Flatten(),
      #then we create our layers
    #.Linear takes an input shape and output shape and then produces a weight and bias term for the specified shape.
      #self.fh1=
      tt.Linear(length_input,length_hidden),
      tt.ReLU(),
      tt.Linear(length_hidden,num_digits),
      #tt.ReLU(),
      #tt.Linear(hidd_2,num_digits),
      #tt.Sigmoid()
    )

    # #use of activation function ReLU
    # self.relu = tt.ReLU()
    # self.fh2=
    # tt.Sigmoid()
    # #sigmoid chosen as the output layer function
  def forward(self,i):
    # self.flat
    # res=self.fh1(i)
    # res=self.relu(res)
    # res=self.fh2(res)
    return self.layers(i)





In [ ]:
module = feedfwdnet(length_input,length_hidden,num_digits)
#selecting the loss function
reg_loss= tt.CrossEntropyLoss()
util = torch.optim.Adam(module.parameters(),lr=learning_rate)

In [ ]:
# for epoch in range(runtime):
#   # Print epoch
#   print(f'Starting epoch {epoch+1}')
#   # Set current loss value
#   current_loss = 0.0
#   # Iterate over the DataLoader for training data
#   for i, data in enumerate(train_loader, 0):
#     # Get inputs
#     inputs, targets = data
#     # Prepare targets
#     targets = targets \
#                  .type(torch.FloatTensor) \
#                  .reshape((targets.shape[0], 1))

#     # Zero the gradients
#     util.zero_grad()

#     # Perform forward pass
#     outputs = module(inputs)

#     # Compute loss
#     loss = reg_loss(outputs, targets.expand_as(outputs))

#     # Perform backward pass
#     loss.backward()

#     # Perform optimization
#     util.step()
n_total_steps = len(train_loader)
for epoch in range(runtime):
  for i, (images, labels) in enumerate(train_loader):
    # origin shape: [100, 1, 28, 28]
    # resized: [100, 784]
    images = images.reshape(-1, 28*28)
    labels = labels
    # Forward pass
    outputs = module(images)
    #labels=torch.as_tensor(labels)
    labels=labels\
               .type(torch.FloatTensor) \
               .reshape((labels.shape[0], 1))
    #labels=torch.tensor([labels,1])
    loss = reg_loss(outputs,labels.expand_as(outputs))
    # Backward and optimize
    util.zero_grad()
    loss.backward()
    util.step()
    # Print statistics
    current_loss += loss.item()
    if i % 10 == 0:
      print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 500))
      current_loss = 0.0

  # Process is complete.
print('Training process has finished.')

Loss after mini-batch     1: 0.507
Loss after mini-batch    11: 0.497
Loss after mini-batch    21: 0.500
Loss after mini-batch    31: 0.500
Loss after mini-batch    41: 0.484
Loss after mini-batch    51: 0.466
Loss after mini-batch    61: 0.505
Loss after mini-batch    71: 0.472
Loss after mini-batch    81: 0.494
Loss after mini-batch    91: 0.482
Loss after mini-batch   101: 0.492
Loss after mini-batch   111: 0.487
Loss after mini-batch   121: 0.482
Loss after mini-batch   131: 0.505
Loss after mini-batch   141: 0.503
Loss after mini-batch   151: 0.490
Loss after mini-batch   161: 0.493
Loss after mini-batch   171: 0.503
Loss after mini-batch   181: 0.480
Loss after mini-batch   191: 0.486
Loss after mini-batch   201: 0.512
Loss after mini-batch   211: 0.497
Loss after mini-batch   221: 0.468
Loss after mini-batch   231: 0.510
Loss after mini-batch   241: 0.490
Loss after mini-batch   251: 0.486
Loss after mini-batch   261: 0.505
Loss after mini-batch   271: 0.492
Loss after mini-batc

In [ ]:

t_ones= torchvision.datasets.MNIST(root=(path/'testing'/'1'),
                                          train=False,
                                          transform=transforms.ToTensor(),download=True)
t_threes = torchvision.datasets.MNIST(root=(path/'testing'/'3'),
                                          train=False,
                                          transform=transforms.ToTensor(),download=True)
t_sevens = torchvision.datasets.MNIST(root=(path/'testing'/'7'),
                                          train=False,
                                          transform=transforms.ToTensor(),download=True)
t_eights = torchvision.datasets.MNIST(root=(path/'testing'/'8'),
                                          train=False,
                                          transform=transforms.ToTensor(),download=True)
# Create joint test set
test_set = t_threes + t_eights + t_ones + t_sevens
#Passing all the Training sets into one
#turning it into tensor format
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=100,shuffle=False)


TESTING THE ACCURACY ON THE TEST SET

In [ ]:
with torch.no_grad():
    correct_num = 0
    samples_num = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = module(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        samples_num += labels.size(0)
        correct_num += (predicted == labels).sum().item()

    accuracy = 100.0 * (correct_num / samples_num)
    print(f'Accuracy of the network: {accuracy} %')

Accuracy of the network: 7.84 %
